In [1]:
import networkx as nx
import igraph as ig
import pickle
import pandas as pd

In [2]:
def load_part(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [3]:
questions = pd.read_csv("../data/cache/prepared_questions_metrics.csv")
answers = pd.read_csv("../data/cache/prepared_answers_metrics.csv", low_memory=False)

In [4]:
tags_graph = ig.Graph.from_networkx(nx.read_edgelist("../data/cache/tagsG.edgelist"))
answers_graph = ig.Graph.from_networkx(nx.read_edgelist("../data/cache/answersG.edgelist"))

In [5]:
tags_part = load_part("../data/cache/user_communities_a.pickle")
answers_part = load_part("../data/cache/user_communities_t.pickle")

In [6]:
def compute(G, partition, questions, answers):
    community_stats = {}

    complex_ids = questions[questions["is_complex"]]["Id"].to_list()

    for community in partition:
        members = G.vs[community]
        total_answers = 0
        total_complex_answers = 0

        unique_answerers = set()
        unique_complex_answerers = set()

        for user in members:
            user_id = user["_nx_name"][1:]
            print(user_id)
            user_answers = answers[answers["OwnerUserId"] == float(user_id)]

            total_answers += len(user_answers)
            unique_answerers.add(user_id)

            complex_user_answers = answers[(answers["OwnerUserId"] == float(user_id)) & (answers["ParentId"].isin(complex_ids))]
            total_complex_answers += len(complex_user_answers)
            if len(complex_user_answers) > 0:
                unique_complex_answerers.add(user_id)

        community_stats[community] = {
            "total_answers": total_answers,
            "total_complex_answers": total_complex_answers,
            "unique_answerers": len(unique_answerers),
            "unique_complex_answerers": len(unique_complex_answerers),
            "complex_to_total_ratio": total_complex_answers / total_answers if total_answers > 0 else 0
        }

    return community_stats

In [ ]:
compute(answers_graph, answers_part, answers_graph, questions)
compute(tags_graph, tags_part, tags_graph, questions)

KeyError: 'Attribute does not exist'